In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 

In [3]:
data=pd.read_csv("Data/finalTrain.csv")

In [4]:
df=data.copy()

In [5]:
#to split the dependent and target features
X=df.iloc[:,:-1]
y=df.iloc[:,-1:]

In [6]:
numerical_cols=X.select_dtypes(exclude='object').columns
categorical_cols=X.select_dtypes(include='object').columns

In [6]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries'],
      dtype='object')

In [7]:
categorical_cols

Index(['ID', 'Delivery_person_ID', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Type_of_order', 'Type_of_vehicle', 'Festival', 'City'],
      dtype='object')

In [8]:
Road_traffic_density_cat=['Low','Medium','High','Jam']

In [7]:
from sklearn.base import BaseEstimator,TransformerMixin

In [ ]:
 distance = distcalculate(df.loc['Restaurant_latitude'][i], 
                                  df['Restaurant_longitude'][i], 
                                  df['Delivery_location_latitude'][i], 
                                  df['Delivery_location_longitude'][i])
        

In [9]:
#Create a pipeline to perform the various steps in data processing and scaling
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.compose import ColumnTransformer

In [10]:
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('haversine',DistanceCalculator()),
    ('scaler', StandardScaler())
])



In [11]:
# Fit and transform the data
X_transformed = num_pipeline.fit_transform(df[numerical_cols])


AttributeError: 'numpy.ndarray' object has no attribute 'loc'

In [27]:
class DistanceCalculator(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        distances = np.empty((len(X), 1))

        for i in range(len(X)):
            distances[i] = self._calculate_distance(X['Restaurant_latitude'][i], 
                                                     X['Restaurant_longitude'][i], 
                                                     X['Delivery_location_latitude'][i], 
                                                     X['Delivery_location_longitude'][i])

        distances_df = pd.DataFrame(distances, columns=['distance'])                                             

        return distances_df
    
    
    # Set the earth's radius (in kilometers)
    R = 6371

    # Convert degrees to radians
    def deg_to_rad(self,degrees):
        return degrees * (np.pi/180)

    # Function to calculate the distance between two points using the haversine formula
    def _calculate_distance(self,lat1, lon1, lat2, lon2):
        d_lat = deg_to_rad(lat2-lat1)
        d_lon = deg_to_rad(lon2-lon1)
        a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        return R * c

# Create pipeline
dist_pipeline = Pipeline([
    
    ('distance_calculator', DistanceCalculator()),
    ('scaler', StandardScaler())
])

# Create column transformer
ct = ColumnTransformer([
    ('drop_cols', 'drop', ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']),
    ('distance', dist_pipeline, ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude'])
])

# Apply column transformer to dataframe
X_transformed = ct.fit_transform(df[numerical_cols])

# Convert transformed data to dataframe
distance_df = pd.DataFrame(X_transformed, columns=['distance'])


In [17]:
X_transformed=pd.DataFrame(X_transformed)
X_transformed

,0
0,-0.080959
1,-0.084631
2,-0.077770
3,-0.087642
4,-0.072670
...,...
45579,-0.088952
45580,-0.080298
45581,-0.086072
45582,-0.084640


In [75]:
class DistanceCalculator(TransformerMixin):
    def __init__(self):
        pass

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        distances = np.empty((len(X), 1))

        for i in range(len(X)):
            distances[i] = self._calculate_distance(X['Restaurant_latitude'][i], 
                                                     X['Restaurant_longitude'][i], 
                                                     X['Delivery_location_latitude'][i], 
                                                     X['Delivery_location_longitude'][i])
         
        distances_df = pd.DataFrame(distances, columns=['distance'])                                             

        return distances_df
    
    
    # Set the earth's radius (in kilometers)
    R = 6371

    # Convert degrees to radians
    def deg_to_rad(self,degrees):
        return degrees * (np.pi/180)

    # Function to calculate the distance between two points using the haversine formula
    def _calculate_distance(self,lat1, lon1, lat2, lon2):
        d_lat = deg_to_rad(lat2-lat1)
        d_lon = deg_to_rad(lon2-lon1)
        a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        return R * c

# Create pipeline
numerical_transformer=Pipeline([
    ('impute',SimpleImputer())
])

column_dropper_transformer=Pipeline([
    ('dropper',column_dropper())
])

dist_pipeline = Pipeline([
    
    ('distance_calculator', DistanceCalculator()),
    ('scaler', StandardScaler())
])

# Create column transformer
ct = ColumnTransformer([
    # ('drop_cols', 'drop', ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']),
    ('imputer',numerical_transformer,numerical_cols),
    ('distance', dist_pipeline, ['Restaurant_latitude', 'Restaurant_longitude', 'Delivery_location_latitude', 'Delivery_location_longitude']),
    
],remainder='passthrough')

# Apply column transformer to dataframe
X_transformed = ct.fit_transform(df[numerical_cols])



# # Convert transformed data to dataframe
# distance_df = pd.DataFrame(X_transformed, columns=['distance'])

In [76]:
X_transformed=pd.DataFrame(X_transformed)

In [65]:
numerical_cols.shape

(8,)

In [77]:
X_transformed

,0,1,2,3,4,5,6,7,8
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,2.0,3.0,-0.080959
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,1.0,1.0,-0.084631
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,1.0,1.0,-0.077770
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,0.0,0.0,-0.087642
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,1.0,1.0,-0.072670
...,...,...,...,...,...,...,...,...,...
45579,30.0,4.8,26.902328,75.794257,26.912328,75.804257,1.0,0.0,-0.088952
45580,21.0,4.6,0.000000,0.000000,0.070000,0.070000,0.0,1.0,-0.080298
45581,30.0,4.9,13.022394,80.242439,13.052394,80.272439,1.0,0.0,-0.086072
45582,20.0,4.7,11.001753,76.986241,11.041753,77.026241,0.0,1.0,-0.084640
